In [1]:
from pandas import read_csv

# The Data
## Overview

2.

(2a)
* What sources - brief description, give some headline assumptions
  * UNPD
  * GBD
  * Earth4All
(2b)
* Scenarios we're focusing on, maybe a plot in here with headline assumptions
  * UNPD: High, Med, Low
  * GBD: Reference, SDG Met Need
  * Earth4All: Too Little Too Late, Giant Leap

## Initial look at the data
3.
(3a)
* Methodology
  * Multiple ways to pull data from UNPD, API or catalogue of downloads, downloading csv simpler for exploratory work (haven't decided what we're looking for) but for specific datasets I've got some simple bits of python to wrap the API and pull in specific datasets
  * At least for UNPD (picture is a little more complex for other forecasts) most of the useful/interesting metrics are either in the data or a relatively simple calculation
  * One big project-specific question is "when are the Next 8 Billion born?"
  * Set methodology here.
  * Note that not all forecasts have a next 8 bn this century - suggest one could do a simple extrapolation but this overlooks the complexity of the forecasts and all the assumptions that will be broken in doing this.
* When do we reach 8 Billion?
  * Plot of cumulative births since 2023
  * Maybe add (faint) lines for specific forecasts?
  * Plot all births as well, make it a births-focused section
* Population stats
  * Total population
  * Median age
  * Dependency ratio
  * Do all of above grouped by income groups
  * G20, P20
  
# Dave wrap up